# Triangle Counting

In this notebook, we will count the numner of trianges in our test dataset.  The NetworkX and cuGraph processes will be interleaved so that each step can be compared.

Notebook Credits
* Original Authors: Bradley Rees
* Created:   08/01/2019
* Last Edit: 10/21/2019

RAPIDS Versions: 0.10.0    

Test Hardware

* GV100 32G, CUDA 10.0


## Introduction
Triancle Counting, as the name implies, finds the number of triangles in a graph.  Triangles are important in computing the clustering Coefficient and can be used for clustering.  


To compute the Pagerank scores for a graph in cuGraph we use:<br>

**cugraph.triangles(G)**
* __G__: cugraph.Graph object


Returns:
* int64 count


## cuGraph Notice 
The current version of cuGraph has some limitations:

* Vertex IDs need to be 32-bit integers.
* Vertex IDs are expected to be contiguous integers starting from 0.

cuGraph provides the renumber function to mitigate this problem. Input vertex IDs for the renumber function can be either 32-bit or 64-bit integers, can be non-contiguous, and can start from an arbitrary number. The renumber function maps the provided input vertex IDs to 32-bit contiguous integers starting from 0. cuGraph still requires the renumbered vertex IDs to be representable in 32-bit integers. These limitations are being addressed and will be fixed soon.    

### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](../img/zachary_black_lines.png)


### Prep

In [1]:
# Import needed libraries
import cugraph
import cudf
from collections import OrderedDict

In [2]:
# NetworkX libraries
import networkx as nx
from scipy.io import mmread

### Some Prep

In [3]:
# Define the path to the test data  
datafile='../data/karate-data.csv'

---
# NetworkX

In [4]:
# Read the data, this also created a NetworkX Graph 
file = open(datafile, 'rb')
Gnx = nx.read_edgelist(file)

In [5]:
nx_count = nx.triangles(Gnx)

In [6]:
# NetworkX does not give a single count, but list how many triangles each vertex is associated with
nx_count

{'1': 18,
 '2': 12,
 '3': 11,
 '4': 10,
 '5': 2,
 '6': 3,
 '7': 3,
 '8': 6,
 '9': 5,
 '11': 2,
 '12': 0,
 '13': 1,
 '14': 6,
 '18': 1,
 '20': 1,
 '22': 1,
 '32': 3,
 '31': 3,
 '10': 0,
 '28': 1,
 '29': 1,
 '33': 13,
 '17': 1,
 '34': 15,
 '15': 1,
 '16': 1,
 '19': 1,
 '21': 1,
 '23': 1,
 '24': 4,
 '26': 1,
 '30': 4,
 '25': 1,
 '27': 1}

In [7]:
# To get the number of triangles, we would need to loop through the array and add up each count
count = 0
for key, value in nx_count.items():
    count = count + value

In [8]:
count

135

Let's seet how that compares to cuGraph

----

# cuGraph

### Read in the data - GPU
cuGraph depends on cuDF for data loading and the initial Dataframe creation

The data file contains an edge list, which represents the connection of a vertex to another.  The `source` to `destination` pairs is in what is known as Coordinate Format (COO).  In this test case, the data is just two columns.  However a third, `weight`, column is also possible

In [9]:
# Test file    
gdf = cudf.read_csv(datafile, delimiter='\t', names=['src', 'dst'], dtype=['int32', 'int32'] )

### Create a Graph 

In [10]:
# create a Graph using the source (src) and destination (dst) vertex pairs from the Dataframe 
G = cugraph.Graph()
G.from_cudf_edgelist(gdf, source='src', destination='dst')

### Call the Triangle Counting 

In [11]:
# Call cugraph.pagerank to get the pagerank scores
cu_count = cugraph.triangles(G)

In [12]:
cu_count

135

_It was that easy!_  

----


___
Copyright (c) 2019, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___